In [6]:
import pandas as pd
from datetime import datetime
df = pd.read_csv('retail_store_inventory_sorted.csv')

if 'Toys' in df['Category'].values or 'Furniture' in df['Category'].values:
    df = df[~df['Category'].isin(['Toys', 'Furniture'])]
    
    counts = df['Category'].value_counts()
    total_to_redistribute = 14643 + 14699  # Toys + Furniture
    total_current = counts.sum()
    proportions = counts / total_current
    new_rows = []
    
    for category, proportion in proportions.items():
        rows_to_add = int(total_to_redistribute * proportion)
        category_rows = df[df['Category'] == category].sample(n=rows_to_add, replace=True, random_state=42)
        new_rows.append(category_rows)
    
    df = pd.concat([df] + new_rows, ignore_index=True)

def assign_season(date):
    date = pd.to_datetime(date)
    month, day = date.month, date.day
    if (month == 2 and day >= 15) or (month == 3) or (month == 4 and day <= 14):
        return 'Spring'
    elif (month == 4 and day >= 15) or (month == 5) or (month == 6 and day <= 14):
        return 'Summer'
    elif (month == 6 and day >= 15) or (month == 7) or (month == 8 and day <= 14):
        return 'Monsoon'
    elif (month == 8 and day >= 15) or (month == 9) or (month == 10 and day <= 14):
        return 'Autumn'
    elif (month == 10 and day >= 15) or (month == 11) or (month == 12 and day <= 14):
        return 'Pre-winter'
    else: 
        return 'Winter'

df['Season'] = df['Date'].apply(assign_season)
df['Category_Type'] = df['Season'] + ' ' + df['Category']

product_names = {
    'Spring Clothing': 'Light Jacket', 'Summer Clothing': 'T-Shirt', 'Monsoon Clothing': 'Raincoat',
    'Autumn Clothing': 'Cardigan', 'Pre-winter Clothing': 'Sweater', 'Winter Clothing': 'Coat',
    'Spring Groceries': 'Strawberries', 'Summer Groceries': 'Ice Cream', 'Monsoon Groceries': 'Instant Noodles',
    'Autumn Groceries': 'Pasta', 'Pre-winter Groceries': 'Soup', 'Winter Groceries': 'Hot Chocolate',
    'Spring Electronics': 'Smartphone', 'Summer Electronics': 'Air Conditioner', 'Monsoon Electronics': 'Waterproof Speaker',
    'Autumn Electronics': 'Laptop', 'Pre-winter Electronics': 'Electric Kettle', 'Winter Electronics': 'Heater'
}

def assign_product_name(row):
    category_type = row['Category_Type']
    current_name = row['Name']
    typical_name = product_names.get(category_type, 'Unknown')
    if current_name in [typical_name, 'Unknown'] or current_name.lower() in typical_name.lower():
        return typical_name
    return current_name

df['Name'] = df.apply(assign_product_name, axis=1)

df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(['Store ID', 'Product ID', 'Date'])
df['Units_Sold_Lag1'] = df.groupby(['Store ID', 'Product ID'])['Units Sold'].shift(1)
df['Units_Sold_Lag7'] = df.groupby(['Store ID', 'Product ID'])['Units Sold'].shift(7)
df['Units_Sold_MA7'] = df.groupby(['Store ID', 'Product ID'])['Units Sold'].rolling(window=7, min_periods=1).mean().reset_index(level=[0,1], drop=True)

df['Inventory_to_Sales'] = df['Inventory Level'] / df['Units Sold'].replace(0, 1)  # Avoid division by zero

for season in ['Spring', 'Summer', 'Monsoon', 'Autumn', 'Pre-winter', 'Winter']:
    df[f'Is_{season}'] = (df['Season'] == season).astype(int)

df = df.drop(columns=['Season'])

df.to_csv('retail_store_inventory_modified.csv', index=False)

In [7]:
import pandas as pd
import numpy as np
df = pd.read_csv('retail_store_inventory_modified.csv')
df_filtered = df[~df['Category'].isin(['Furniture', 'Toys'])].copy()
category_mappings = {
    'Clothing': {
        'Summer': ['T-Shirts', 'Shorts', 'Sunglasses'],
        'Winter': ['Jackets', 'Sweaters', 'Scarves'],
        'Rainy': ['Raincoats', 'Umbrellas', 'Waterproof Boots'],
        'Autumn': ['Hoodies', 'Cardigans', 'Jeans']
    },
    'Groceries': {
        'Summer': ['Cold Drinks', 'Ice Cream', 'Fruits'],
        'Winter': ['Soups', 'Canned Food', 'Hot Chocolate'],
        'Rainy': ['Tea', 'Snacks', 'Instant Noodles'],
        'Autumn': ['Pasta', 'Sauces', 'Spices']
    },
    'Electronics': {
        'Summer': ['Fans', 'ACs', 'Coolers'],
        'Winter': ['Heaters', 'Electric Blankets', 'Geysers'],
        'Rainy': ['Waterproof Speakers', 'Smartphones', 'Power Banks'],
        'Autumn': ['Laptops', 'Tablets', 'Smartwatches']
    }
}

def assign_category_info(row):
    season = row['Seasonality']
    category = row['Category']
    if season in category_mappings[category]:
        product_list = category_mappings[category][season]
        product = np.random.choice(product_list)
        category_type = f"{season} {category}"
        return pd.Series([category_type, product])
    else:
        return pd.Series([f"{season} {category}", "Unknown"])
df_filtered[['Category_Type', 'Name']] = df_filtered.apply(assign_category_info, axis=1)

df_filtered[['Category', 'Seasonality', 'Category_Type', 'Name']].head()

df = pd.read_csv('retail_store_inventory.csv')
df_filtered = df[~df['Category'].isin(['Furniture', 'Toys'])].copy()
removed_count = len(df[df['Category'].isin(['Furniture', 'Toys'])])
new_rows = []
stores = df_filtered['Store ID'].unique()
regions = df_filtered['Region'].unique()
weather_conditions = df_filtered['Weather Condition'].unique()
holidays = [0, 1]
seasons = ['Summer', 'Winter', 'Rainy', 'Autumn', 'Spring']
categories = ['Clothing', 'Groceries', 'Electronics']

for _ in range(removed_count):
    season = np.random.choice(seasons)
    category = np.random.choice(categories)
    
    if season in category_mappings.get(category, {}):
        name = np.random.choice(category_mappings[category][season])
    else:
        name = "Unknown"
    
    new_rows.append({
        'Date': np.random.choice(df_filtered['Date']),
        'Store ID': np.random.choice(stores),
        'Product ID': f"PX{np.random.randint(1000, 9999)}",
        'Category': category,
        'Region': np.random.choice(regions),
        'Inventory Level': np.random.randint(50, 500),
        'Units Sold': np.random.randint(10, 300),
        'Units Ordered': np.random.randint(10, 300),
        'Demand Forecast': round(np.random.uniform(50.0, 200.0), 2),
        'Price': round(np.random.uniform(10.0, 100.0), 2),
        'Discount': np.random.choice([0, 10, 20, 30]),
        'Weather Condition': np.random.choice(weather_conditions),
        'Holiday/Promotion': np.random.choice(holidays),
        'Competitor Pricing': round(np.random.uniform(10.0, 100.0), 2),
        'Seasonality': season,
        'Category_Type': f"{season} {category}",
        'Name': name
    })

# Convert to DataFrame
df_synthetic = pd.DataFrame(new_rows)

# Combine with original filtered dataset
df_final = pd.concat([df_filtered, df_synthetic], ignore_index=True)

# Save to CSV
output_path = "retail_store_inventory_updated.csv"
df_final.to_csv(output_path, index=False)

output_path

'retail_store_inventory_updated.csv'

In [9]:
import pandas as pd
data = pd.read_csv('retail_store_inventory_modified.csv')
data[['Units_Sold_Lag1', 'Units_Sold_Lag7']] = data.groupby(['Store ID', 'Product ID'])[['Units_Sold_Lag1', 'Units_Sold_Lag7']].fillna(method='ffill')
data[['Units_Sold_Lag1', 'Units_Sold_Lag7']] = data[['Units_Sold_Lag1', 'Units_Sold_Lag7']].fillna(0)
print(data[['Units_Sold_Lag1', 'Units_Sold_Lag7']].isnull().sum())
data.to_csv('retail_store_inventory_modified.csv', index=False)
validation = data.groupby(['Store ID', 'Product ID'])[['Units_Sold_Lag1', 'Units_Sold_Lag7']].max()
print(validation)


C:\Users\aryan\AppData\Local\Temp\ipykernel_21536\2647304764.py:3: FutureWarning: DataFrameGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use DataFrame.fillna instead
  data[['Units_Sold_Lag1', 'Units_Sold_Lag7']] = data.groupby(['Store ID', 'Product ID'])[['Units_Sold_Lag1', 'Units_Sold_Lag7']].fillna(method='ffill')
C:\Users\aryan\AppData\Local\Temp\ipykernel_21536\2647304764.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data[['Units_Sold_Lag1', 'Units_Sold_Lag7']] = data.groupby(['Store ID', 'Product ID'])[['Units_Sold_Lag1', 'Units_Sold_Lag7']].fillna(method='ffill')


Units_Sold_Lag1    0
Units_Sold_Lag7    0
dtype: int64
                     Units_Sold_Lag1  Units_Sold_Lag7
Store ID Product ID                                  
S001     P0001                 461.0            461.0
         P0002                 486.0            486.0
         P0003                 484.0            484.0
         P0004                 475.0            475.0
         P0005                 477.0            477.0
...                              ...              ...
S005     PX9985                  0.0              0.0
         PX9989                  0.0              0.0
         PX9993                248.0              0.0
         PX9995                202.0              0.0
         PX9996                 83.0              0.0

[21660 rows x 2 columns]


In [10]:
import pandas as pd
df = pd.read_csv('retail_store_inventory_cleaned.csv')
df['Date'] = pd.to_datetime(df['Date'])
df_sorted = df.sort_values(by='Date')
df_sorted.to_csv('retail_store_inventory_sorted.csv', index=False)
print(df_sorted.head())

            Date Store ID Product ID     Category Region  Inventory Level  \
0     2022-01-01     S001      P0001    Groceries  North              231   
17202 2022-01-01     S002      P0007    Groceries   West              460   
17644 2022-01-01     S002      P0008  Electronics   West              304   
7845  2022-01-01     S001      P0019     Clothing   East              352   
18100 2022-01-01     S002      P0009     Clothing  South              113   

       Units Sold  Units Ordered  Demand Forecast  Price  ...  \
0             127             55           135.47  33.50  ...   
17202         393             70           401.48  91.13  ...   
17644          29             99            35.54  66.96  ...   
7845          257            186           267.38  73.28  ...   
18100           9             84            14.78  81.29  ...   

       Units_Sold_Lag1 Units_Sold_Lag7  Units_Sold_MA7  Inventory_to_Sales  \
0                  0.0             0.0           127.0            1.

In [ ]:
import pandas as pd

# Load the dataset
data = pd.read_csv('retail_store_inventory.csv')

# Assuming the category column is named 'Category'
category_counts = data['Category'].value_counts()

print(category_counts)


In [11]:
import pandas as pd
data = pd.read_csv('retail_store_inventory_modified.csv')
category_counts = data['Category_Type'].value_counts()

print(category_counts)

Category_Type
Winter Electronics        4173
Summer Groceries          4170
Summer Clothing           4165
Winter Groceries          4163
Autumn Clothing           4113
Winter Clothing           4100
Monsoon Electronics       4099
Pre-winter Groceries      4096
Pre-winter Electronics    4092
Monsoon Clothing          4076
Autumn Electronics        4076
Pre-winter Clothing       4031
Autumn Groceries          4013
Summer Electronics        4010
Spring Groceries          3996
Monsoon Groceries         3972
Spring Clothing           3917
Spring Electronics        3838
Name: count, dtype: int64


In [12]:
import pandas as pd
data = pd.read_csv('retail_store_inventory_modified.csv')
category_counts = data['Name'].value_counts()

print(category_counts)

Name
Hot Chocolate          2879
Ice Cream              2840
Pasta                  2800
Spices                 1960
Coolers                1927
Fans                   1909
Cardigans              1902
Laptops                1897
Soups                  1895
Sweaters               1895
Geysers                1892
Shorts                 1888
Cold Drinks            1886
Jackets                1884
T-Shirts               1877
ACs                    1873
Electric Blankets      1863
Scarves                1857
Fruits                 1853
Hoodies                1850
Sunglasses             1833
Sauces                 1827
Smartwatches           1825
Heaters                1809
Jeans                  1804
Canned Food            1803
Tablets                1790
Instant Noodles        1548
T-Shirt                1032
Sweater                 971
Cardigan                970
Electric Kettle         967
Heater                  959
Air Conditioner         953
Light Jacket            938
Laptop         

In [13]:
import pandas as pd

df = pd.read_csv('retail_store_inventory_modified.csv')
category_mappings = {
    'Clothing': {
        'Summer': ['T-Shirts', 'Shorts', 'Sunglasses'],
        'Winter': ['Jackets', 'Sweaters', 'Scarves'],
        'Rainy': ['Raincoats', 'Umbrellas', 'Waterproof Boots'],
        'Autumn': ['Hoodies', 'Cardigans', 'Jeans']
    },
    'Groceries': {
        'Summer': ['Cold Drinks', 'Ice Cream', 'Fruits'],
        'Winter': ['Soups', 'Canned Food', 'Hot Chocolate'],
        'Rainy': ['Tea', 'Snacks', 'Instant Noodles'],
        'Autumn': ['Pasta', 'Sauces', 'Spices']
    },
    'Electronics': {
        'Summer': ['Fans', 'ACs', 'Coolers'],
        'Winter': ['Heaters', 'Electric Blankets', 'Geysers'],
        'Rainy': ['Waterproof Speakers', 'Smartphones', 'Power Banks'],
        'Autumn': ['Laptops', 'Tablets', 'Smartwatches']
    }
}

product_names = {
    'Spring Clothing': 'Light Jacket', 'Summer Clothing': 'T-Shirt', 'Monsoon Clothing': 'Raincoat',
    'Autumn Clothing': 'Cardigan', 'Pre-winter Clothing': 'Sweater', 'Winter Clothing': 'Coat',
    'Spring Groceries': 'Strawberries', 'Summer Groceries': 'Ice Cream', 'Monsoon Groceries': 'Instant Noodles',
    'Autumn Groceries': 'Pasta', 'Pre-winter Groceries': 'Soup', 'Winter Groceries': 'Hot Chocolate',
    'Spring Electronics': 'Smartphone', 'Summer Electronics': 'Air Conditioner', 'Monsoon Electronics': 'Waterproof Speaker',
    'Autumn Electronics': 'Laptop', 'Pre-winter Electronics': 'Electric Kettle', 'Winter Electronics': 'Heater'
}

def extract_category_season(cat_type):
    parts = cat_type.split()
    if len(parts) == 2:
        season, category = parts
    elif len(parts) == 3:  
        season = 'Pre-winter'
        category = parts[2]
    else:
        season, category = None, None
    return category, season

def map_product(row):
    cat_type = row['Category_Type']
    name = row['Name']
    category, season = extract_category_season(cat_type)
    canonical_name = product_names.get(cat_type, name)
    mapped_category = None
    mapped_season = None
    for cat, seasons in category_mappings.items():
        for seas, products in seasons.items():
            if name in products:
                mapped_category = cat
                mapped_season = seas
    return pd.Series([category, season, canonical_name, mapped_category, mapped_season])

df[['Extracted_Category', 'Extracted_Season', 'Canonical_Name', 'Mapped_Category', 'Mapped_Season']] = df.apply(map_product, axis=1)
df.to_csv('retail_store_inventory_with_mappings.csv', index=False)


In [14]:
import pandas as pd

# Load your dataset
df = pd.read_csv('retail_store_inventory_modified.csv')

# Your product_names mapping
product_names = {
    'Spring Clothing': 'Light Jacket', 'Summer Clothing': 'T-Shirt', 'Monsoon Clothing': 'Raincoat',
    'Autumn Clothing': 'Cardigan', 'Pre-winter Clothing': 'Sweater', 'Winter Clothing': 'Coat',
    'Spring Groceries': 'Strawberries', 'Summer Groceries': 'Ice Cream', 'Monsoon Groceries': 'Instant Noodles',
    'Autumn Groceries': 'Pasta', 'Pre-winter Groceries': 'Soup', 'Winter Groceries': 'Hot Chocolate',
    'Spring Electronics': 'Smartphone', 'Summer Electronics': 'Air Conditioner', 'Monsoon Electronics': 'Waterproof Speaker',
    'Autumn Electronics': 'Laptop', 'Pre-winter Electronics': 'Electric Kettle', 'Winter Electronics': 'Heater'
}

# Update the Name column based on Category_Type
df['Name'] = df['Category_Type'].map(product_names).fillna(df['Name'])

# Save the updated DataFrame if needed
df.to_csv('retail_store_inventory_modified_updated.csv', index=False)


In [19]:
import pandas as pd

# Replace 'your_dataset.csv' with your actual file name
df = pd.read_csv('retail_store_inventory_modified_2.csv')

def map_season(season):
    season = season.lower()
    if 'summer' in season:
        return 'summer'
    elif 'winter' in season:
        # Check for 'pre-winter' first to avoid mislabeling
        if 'pre-winter' in season:
            return 'pre-winter'
        return 'winter'
    elif 'pre-winter' in season:
        return 'pre-winter'
    elif 'monsoon' in season:
        return 'monsoon'
    elif 'spring' in season:
        return 'spring'
    else:
        return season  # Keep as is if not matched

df['Season'] = df['Season'].apply(map_season)

# Save the updated DataFrame if needed
df.to_csv('retail_store_inventory_modified_2.csv', index=False)
